## Webscraping para obtener datos de narcotraficantes en México

Para realizar esto, se utilizarán los datos de Wikipedia en español. Se obtendrán los nombres de los narcotraficantes y se buscará información adicional como su lugar de nacimiento y el cartel al que pertenecen.

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
page = requests.get("https://es.wikipedia.org/wiki/Categor%C3%ADa:Narcotraficantes_de_M%C3%A9xico")
soup = BeautifulSoup(page.content, 'html.parser')
results=soup.find_all('div', class_='mw-category-group')

In [3]:
#Obtener nombres y urls
narco = [a.get_text() for div in results for a in div.find_all('a')]
urls=[a.get('href') for div in results for a in div.find_all('a')]

In [4]:
tabla_gral=pd.DataFrame({'narco':narco,'url':urls})
#Añadir https://es.wikipedia.org/ a las urls
tabla_gral['url']="https://es.wikipedia.org"+tabla_gral['url']
#Eliminar registro 7
tabla_gral.drop(7, inplace=True)
tabla_gral

,narco,url
0,Familia Beltrán Leyva,https://es.wikipedia.org/wiki/Categor%C3%ADa:F...
1,Familia Cárdenas Guillén,https://es.wikipedia.org/wiki/Categor%C3%ADa:F...
2,Familia Coronel,https://es.wikipedia.org/wiki/Categor%C3%ADa:F...
3,Familia Guzmán,https://es.wikipedia.org/wiki/Categor%C3%ADa:F...
4,Familia Oseguera,https://es.wikipedia.org/wiki/Categor%C3%ADa:F...
...,...,...
60,Manuel Torres Félix,https://es.wikipedia.org/wiki/Manuel_Torres_F%...
61,Édgar Valdez Villarreal,https://es.wikipedia.org/wiki/%C3%89dgar_Valde...
62,Érick Valencia Salazar,https://es.wikipedia.org/wiki/%C3%89rick_Valen...
63,Sergio Villarreal Barragán,https://es.wikipedia.org/wiki/Sergio_Villarrea...


In [5]:
#Tabla familias
tabla_familias=tabla_gral.iloc[1:8]
#Tabla nombres
tabla_nombres=tabla_gral.iloc[8:]

In [6]:
#Obtener los nombres de los narcotraficantes de quienes están agrupados por familia
nombres_restantes = pd.DataFrame()

for i in range(tabla_familias.shape[0]):
    page = requests.get(tabla_familias.iloc[i, 1])
    soup = BeautifulSoup(page.content, 'html.parser')
    results = soup.find_all('div', class_='mw-category-group')
    
    narco = [a.get_text() for div in results for a in div.find_all('a')]
    urls = [a.get('href') for div in results for a in div.find_all('a')]

    tabla = pd.DataFrame({'narco': narco, 'url': urls})
    tabla['url'] = tabla['url'].astype(str)

    tabla['url'] = "https://es.wikipedia.org" + tabla['url']
    
    nombres_restantes = pd.concat([nombres_restantes, tabla], axis=0)

nombres_restantes.reset_index(drop=True, inplace=True)


In [7]:
#Pegar nombres_restantes a tabla_nombres
tabla_nombres=pd.concat([tabla_nombres,nombres_restantes],axis=0)
tabla_nombres.reset_index(drop=True, inplace=True)
tabla_nombres

,narco,url
0,Rafael Aguilar Guajardo,https://es.wikipedia.org/wiki/Rafael_Aguilar_G...
1,Benjamín Arellano Félix,https://es.wikipedia.org/wiki/Benjam%C3%ADn_Ar...
2,Enedina Arellano Félix,https://es.wikipedia.org/wiki/Enedina_Arellano...
3,Francisco Javier Arellano Félix,https://es.wikipedia.org/wiki/Francisco_Javier...
4,Francisco Rafael Arellano Félix,https://es.wikipedia.org/wiki/Francisco_Rafael...
...,...,...
69,Rubén Oseguera González,https://es.wikipedia.org/wiki/Rub%C3%A9n_Osegu...
70,Miguel Treviño Morales,https://es.wikipedia.org/wiki/Miguel_Trevi%C3%...
71,Omar Treviño Morales,https://es.wikipedia.org/wiki/Omar_Trevi%C3%B1...
72,Ismael Zambada García,https://es.wikipedia.org/wiki/Ismael_Zambada_G...


### Obtener información adicional   

In [8]:
# Añadir columnas de nacimiento y cartel a la tabla
tabla_nombres['nacimiento'] = None
tabla_nombres['cartel'] = None

# Loop para obtener información adicional
for index, row in tabla_nombres.iterrows():
    url = row['url']
    try:

        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')

        # Extraer fecha de nacimiento
        nacimiento_row = soup.find('th', string="Nacimiento")
        if nacimiento_row:
            nacimiento_data = nacimiento_row.find_next('td').get_text(strip=True)
        else:
            nacimiento_data = None
        tabla_nombres.at[index, 'nacimiento'] = nacimiento_data

        # Extraer cartel
        miembro_row = soup.find('th', string="Miembro de") or soup.find('th', string="Empleador")
        if miembro_row:
            
            cartel_data = miembro_row.find_next('td').get_text(strip=True)
        else:
            cartel_data = None
        tabla_nombres.at[index, 'cartel'] = cartel_data
        
        

    except Exception as e:
        tabla_nombres.at[index, 'nacimiento'] = f"Error: {str(e)}"
        tabla_nombres.at[index, 'cartel'] = f"Error: {str(e)}"


In [9]:
#Tabla final
tabla_nombres

,narco,url,nacimiento,cartel
0,Rafael Aguilar Guajardo,https://es.wikipedia.org/wiki/Rafael_Aguilar_G...,1955México,Cartel de JuárezDirección Federal de Seguridad
1,Benjamín Arellano Félix,https://es.wikipedia.org/wiki/Benjam%C3%ADn_Ar...,"12 de marzo de 1952 (72 años)Culiacán,Sinaloa,...",Cártel de Tijuana
2,Enedina Arellano Félix,https://es.wikipedia.org/wiki/Enedina_Arellano...,"12 de abril de 1961 (63 años)Mazatlán,Sinaloa,...",None
3,Francisco Javier Arellano Félix,https://es.wikipedia.org/wiki/Francisco_Javier...,"11 de diciembre de 1969 (55 años)Culiacán,Sina...",Cártel de Tijuana
4,Francisco Rafael Arellano Félix,https://es.wikipedia.org/wiki/Francisco_Rafael...,"24 de octubre de 1949Culiacán,Sinaloa,México",Cártel de Tijuana
...,...,...,...,...
69,Rubén Oseguera González,https://es.wikipedia.org/wiki/Rub%C3%A9n_Osegu...,"14 de febrero de 1990 (34 años)San Francisco,C...",Cártel de Jalisco Nueva Generación
70,Miguel Treviño Morales,https://es.wikipedia.org/wiki/Miguel_Trevi%C3%...,"18 de noviembre de 1970 (54 años)Nuevo Laredo,...",None
71,Omar Treviño Morales,https://es.wikipedia.org/wiki/Omar_Trevi%C3%B1...,"26 de enero de 1974 (50 años)Nuevo Laredo,Tama...",Los Zetas
72,Ismael Zambada García,https://es.wikipedia.org/wiki/Ismael_Zambada_G...,"1 de enero de 1948(76 años)El Álamo(Sinaloa,Mé...",Cártel de Sinaloa


In [10]:
#Número total de registros
print("Número total de registros:",tabla_nombres.shape[0])
#Porcentaje de nulos en cartel
print("Porcentaje de nulos en cartel:",tabla_nombres['cartel'].isnull().sum()/len(tabla_nombres)*100)

Número total de registros: 74
Porcentaje de nulos en cartel: 40.54054054054054
